# [NTDS'19] assignment 1: network science
[ntds'19]: https://github.com/mdeff/ntds_2019

[Clément Vignac](https://people.epfl.ch/clement.vignac), [EPFL LTS4](https://lts4.epfl.ch) and
[Guillermo Ortiz Jiménez](https://gortizji.github.io), [EPFL LTS4](https://lts4.epfl.ch).

## Students

* Team: `<your team number>`
* Students: `<your name`> (for the indivudual submission) or `<the name of all students in the team>` (for the team submission)

## Rules

Grading:
* The first deadline is for individual submissions. The second deadline is for the team submission.
* All team members will receive the same grade based on the team solution submitted on the second deadline.
* As a fallback, a team can ask for individual grading. In that case, solutions submitted on the first deadline are graded.
* Collaboration between team members is encouraged. No collaboration between teams is allowed.

Submission:
* Textual answers shall be short. Typically one to two sentences.
* Code has to be clean.
* You cannot import any other library than we imported.
  Note that Networkx is imported in the second section and cannot be used in the first.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart Kernel and Run All Cells" in Jupyter.

## Objective

*To be completed*

# Graph Signal Processing

In this part of the assignment we are going to familiarize ourselves with the main concepts in Graph Signal Processing. 

You can only use the following libraries for this part of the assignment.

In [ ]:
from pygsp import graphs
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
%matplotlib inline

We start by importing the graph that will serve as support for our signals. In this exercise we will use a nearest-neighbor graph constructed from the Stanford Bunny point cloud included in the PyGSP library.

In [ ]:
G = graphs.Bunny()
adjacency = np.asarray(G.W.todense())
n_nodes = adjacency.shape[0]

In [ ]:
def plot_bunny(signal=None,title=''):
    fig = plt.gcf()
    ax = plt.gca()
    if not isinstance(ax, Axes3D):
        ax = plt.subplot(111, projection='3d')
    if signal is not None:
        signal = np.squeeze(signal)
    p = ax.scatter(G.coords[:,0], G.coords[:,1], G.coords[:,2], c=signal, marker='o', s=5, cmap='RdBu_r')
    ax.view_init(elev=-90,azim=90)
    ax.dist = 7
    ax.set_axis_off()
    ax.set_title(title)
    if signal is not None:
        fig.colorbar(p)

In [ ]:
plt.subplot(111,projection='3d')
plot_bunny()

## Exercise 1

Let us start by constructing the combinatorial and normalized graph laplacians from an adjacency matrix.

In [ ]:
def combinatorial_laplacian(adjacency):
    D = np.diag(np.sum(adjacency, 1)) # Degree matrix
    return D - adjacency

def normalized_laplacian(adjacency):
    D_norm = np.diag(np.sum(adjacency, 1)**(-1/2)) 
    laplacian_combinatorial =  combinatorial_laplacian(adjacency)
    return D_norm @ laplacian_combinatorial @ D_norm

In [ ]:
laplacian_comb = combinatorial_laplacian(adjacency)
laplacian_norm = normalized_laplacian(adjacency)

## Exercise 2

Find the spectral decomposition of the two laplacians. Make sure that the eigenvalues and eigenvectors are in increasing order $\lambda_0\leq \lambda_1\leq \dots \leq \lambda_n$

In [ ]:
def spectral_decomposition(laplacian):
    return np.linalg.eigh(laplacian)

In [ ]:
e_comb, U_comb = spectral_decomposition(laplacian_comb)
e_norm, U_norm = spectral_decomposition(laplacian_norm)

Plot the eigenvalues

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(121,)
plt.plot(e_comb)
plt.title('Eigenvalues $L_{comb}$')
plt.subplot(122)
plt.plot(e_norm)
plt.title('Eigenvalues $L_{norm}$')

What do you observe? Can you say what is the relationship between the two plots?

Let us focus now on the normalized laplacian.

In [ ]:
laplacian = laplacian_norm
U = U_norm
e = e_norm

To make things more clear we will plot some of its eigenvectors (0, 1, 3, 10, 100) as signals on the bunny graph.

In [ ]:
plt.figure(figsize=(18, 9))
plt.subplot(231, projection='3d')
plot_bunny(signal=U[:,0], title='Eigenvector #0')
plt.subplot(232, projection='3d')
plot_bunny(signal=U[:,1], title='Eigenvector #1')
plt.subplot(233, projection='3d')
plot_bunny(signal=U[:,2], title='Eigenvector #2')

plt.subplot(234, projection='3d')
plot_bunny(signal=U[:,3], title='Eigenvector #3')
plt.subplot(235, projection='3d')
plot_bunny(signal=U[:,10], title='Eigenvector #10')
plt.subplot(236, projection='3d')
plot_bunny(signal=U[:,100], title='Eigenvector #100')

What can you say in terms of the variations/smoothness of these signals.

If we repeat this experiment with the combinatorial laplacian, can we expect the same behaviour? Explain.

## Exercise 3

Create a function to compute the Graph Fourier Transform and its inverse of a graph signal. **Note**: You can assume that you have internal access to the eigendecomposition (`U` and `e`) of the laplacian.

In [ ]:
def GFT(x):
    return U.T @ x

def iGFT(x):
    return U @ x

Now, let's create an all-pass graph signal

In [ ]:
signal = iGFT(np.ones([n_nodes, 1]))
plot_bunny(signal)

and plot its graph spectrum

In [ ]:
plt.stem(e, GFT(signal), use_line_collection=True)

We will filter this signal using spectral templates. Let us start by creating three templates

In [ ]:
template_lp = np.ones(signal.shape)
template_bp = np.ones(signal.shape)
template_hp = np.ones(signal.shape)

template_lp[e >= 0.1] = 0 # Low-pass filter with cut-off at lambda=0.1
template_bp[e < 0.1] = 0 # Band-pass filter with cut-offs at lambda=0.1 and lambda=0.5
template_bp[e > 0.5] = 0
template_hp[e <= 1] = 0 # High-pass filter with cut-off at lambda=1

Create a function to filter a signal given a specific spectral template

In [ ]:
def template_graph_filter(signal, template):
    signal_gft = GFT(signal)
    filter_gft = signal_gft * template
    return iGFT(filter_gft)

Let us visualize the results

In [ ]:
signal_lp = template_graph_filter(signal,template_lp)
signal_bp = template_graph_filter(signal,template_bp)
signal_hp = template_graph_filter(signal,template_hp)

plt.figure(figsize=(18, 4))
plt.subplot(131, projection='3d')
plot_bunny(signal=signal_lp, title='Low-pass signal')
plt.subplot(132, projection='3d')
plot_bunny(signal=signal_bp, title='Band-pass signal')
plt.subplot(133, projection='3d')
plot_bunny(signal=signal_hp, title='High-pass signal')

What can you say in terms of the signal variations? How would you link to the observations you made before about the spectral decomposition of the laplacian?

## Exercise 4

We have seen how we can use the GFT to define different filters that enhance or reduce certain frequency bands. However, to do so, we require an explicit eigendecomposition of the graph laplacian. For very large graphs this is very intense computationally. We will now see how we can obtain similar results by filtering the signals directly in the vertex domain.

The key idea is to use a polynomial of the graph laplacian to define a graph filter, i.e., $g(L)x=\sum_{k=1}^K \alpha_k L^k x$, and use the fact that the powers of a diagonalizable matrix can be written in terms of powers of its eigenvalues. This is
$$
\begin{equation}
        L^k=(U\Lambda U^T)^k=U\Lambda^k U^T = U\begin{bmatrix}
        \lambda_0^k &\dots & 0\\
        \vdots & \ddots & \vdots\\
        0 & \dots & \lambda_N
        \end{bmatrix} U^T
    \end{equation}
$$

This means, that a polynomial of the graph laplacian acts independently on each eigenvalue of the graph, and has a frequency spectrum of
$$g(\lambda)=\sum_{k=1}^K \alpha_k \lambda^k$$
Hence,
$$g(L)x=\sum_{k=1}^K \alpha_k L^k x=\sum_{k=1}^K \alpha_k U\Lambda^k U^T x=U \left(\sum_{k=1}^K \alpha_k\Lambda^k \right)U^T x=\operatorname{iGFT}\left(g(\Lambda)\operatorname{GFT}(x)\right)$$


With these ingredients, we have reduced the design of graph filters in the vertex domain to a regression task that approximates a given spectral response by a polynomial. There are multiple ways to do this, but in this assignment we will implement a very simple strategy based on [least-squares regression](https://en.wikipedia.org/wiki/Polynomial_regression#Matrix_form_and_calculation_of_estimates).

Let us create a band-pass universal spectral template

In [ ]:
freq_grid = np.linspace(0,2, 200)

template_bp = np.ones(freq_grid.shape)
template_bp[freq_grid < 0.1] = 0
template_bp[freq_grid >= 0.5] = 0

Implement a function to find the coefficients of a polynomial that approximates a given spectral template. **Hint:** `np.vander` and `np.linalg.lstsq`

In [ ]:
def fit_polynomial(freq_grid, order, template):
    A = np.vander(freq_grid, order, increasing=True)
    coeff = np.linalg.lstsq(A, template, rcond=None)[0]
    return coeff

Implement a function to compute the frequency response of that filter

In [ ]:
def fir_graph_filter_response(coeff, freq_grid):
    g = np.zeros_like(freq_grid)
    for n, c in enumerate(coeff):
        g += c * (freq_grid**n)
    return g

Let us fit the band-pass template with several polynomials of different order

In [ ]:
plt.plot(freq_grid, template_bp)
for order in [5, 10, 20, 30]:    
    coeff_bp = fit_polynomial(freq_grid, order, template_bp)
    plt.plot(freq_grid, fir_graph_filter_response(coeff_bp, freq_grid))

Based on the previous plot, choose a filter order that achieves (in your opinion) a good tradeoff in terms of computational complexity and response accuracy.

In [ ]:
order = 10

In [ ]:
coeff_bp = fit_polynomial(freq_grid, order, template_bp)

So far, we have only defined a way to compute the coefficients of our laplacian polynomial. Let us now compute our graph filter.

In [ ]:
def fir_graph_filter(coeff, laplacian):
    g = 0
    for n, c in enumerate(coeff):
        g += c * np.linalg.matrix_power(laplacian, n)
    return g

In [ ]:
g_bp = fir_graph_filter(coeff_bp, laplacian)

Filter the previous all-pass signal with this filter

In [ ]:
signal_bp_fir = g_bp @ signal

Let us compare with the previous version

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(121, projection='3d')
plot_bunny(signal_bp, title='Spectral template')
plt.subplot(122, projection='3d')
plot_bunny(signal_bp_fir, title='FIR filter')

To better compare these signals, let us plot their spectrums

In [ ]:
plt.stem(e, GFT(signal_bp), use_line_collection=True, linefmt='C0', markerfmt='C0')
plt.stem(e, GFT(signal_bp_fir), use_line_collection=True, linefmt='C1', markerfmt='C1')

# Machine learning on Graphs

So far, we have only played with toy examples. Let us see the use of these tools in practice! In particular, let us see how we can use some graph filters to construct features to feed a classifier. For this part of the assignment we will import some extra packages.

In [ ]:
import networkx as nx

import time
import torch
import torch.nn as nn
import torch.nn.functional as F

import dgl.function as fn
from dgl import DGLGraph
from dgl.data.citation_graph import load_cora
from sklearn.linear_model import LogisticRegression

np.random.seed(0)
torch.manual_seed(1)

We will use the CORA dataset and the citation graph that we created in Assignment 1. However, to simplify the next tasks we will directly use the preprocessed version of this dataset contained within the Deep Graph Library.

In [ ]:
cora = load_cora()

features = torch.FloatTensor(cora.features)
labels = torch.LongTensor(cora.labels)

train_mask = torch.BoolTensor(cora.train_mask)
val_mask = torch.BoolTensor(cora.val_mask)
test_mask = torch.BoolTensor(cora.test_mask)

in_feats = features.shape[1]
n_classes = cora.num_labels
n_edges = cora.graph.number_of_edges()

graph = cora.graph
adjacency = np.asarray(nx.to_numpy_matrix(graph))

For this exercise we will use the combinatorial laplacian

In [ ]:
laplacian = combinatorial_laplacian(adjacency)
U, e = spectral_decomposition(laplacian)
e_max = np.max(e)
freq_grid = np.linspace(0, e_max)

## Exercise 5

In this assignment, we will interpret CORA's features as multidimensional graph signals living on the citation graph. Our task is to design a classifier that using these features and the geometry of the graph can identify the type of paper each node represents.

We will start using a classical machine learning approach, and design some hand-crafted features to feed a logistic regression model. We will use GSP to do this.

Based on what you learned in class, your past experience, and your intuition, design a graph spectral template that you believe could enhance important features of the graph. 

**Note 1:** You just need to design one graph filter that we will apply to all features individually. 

**Note 2:** Finding the right filter can be very challenging, don't worry if you can't find it. Just make sure you experiment with a few configurations and parameters.

In [ ]:
filter_template = np.ones(freq_grid.shape)

filter_template[freq_grid >= 0.3] = 0 # Low-pass filter with cut-off at lambda=0.1

Choose a filter order to approximate your filter using laplacian polynomials.

In [ ]:
order = 5

coeff = fit_polynomial(freq_grid, order, filter_template)
graph_filter = fir_graph_filter(coeff, laplacian)

Let's plot the frequency response of your spectral template and the polynomial approximation

In [ ]:
plt.plot(freq_grid, filter_template)
plt.plot(freq_grid, fir_graph_filter_response(coeff, freq_grid))

Now, let's create the new features

In [ ]:
filtered_features = graph_filter @ features.numpy()

To train our classifier we will select a few nodes in our graph for training and fit a [logistic regression classifier](https://en.wikipedia.org/wiki/Logistic_regression) on them. To avoid overfitting to the test set when we do hyperparameter tuning, we will also select a validation set. And finally, we will test our classifier on the rest of the nodes.

In [ ]:
train_features = filtered_features[train_mask,:]
train_labels = labels[train_mask]

val_features = filtered_features[val_mask,:]
val_labels = labels[val_mask]

test_features = filtered_features[test_mask,:]
test_labels = labels[test_mask]

Train a logistic regression classifier. Remember to play with the regularization parameters to achieve a well performing model. **Hint:** Use `sklearn.linear_model.LogisticRegression`.

In [ ]:
log_reg = LogisticRegression(penalty='l2', multi_class="auto", solver="liblinear", C=1e4, fit_intercept=False, max_iter=1000)
log_reg.fit(train_features, train_labels)

Let's evaluate your model...

In [ ]:
train_acc = log_reg.score(train_features, train_labels)
val_acc = log_reg.score(val_features, val_labels)
test_acc = log_reg.score(test_features, test_labels)

print('Train accuracy {:.4f} | Validation accuracy {:.4f} | Test accuracy {:.4f}'.format(train_acc, val_acc, test_acc))

## Exercise 6

By now, you will probably have seen that it is very difficult to find the right combination of spectral response, filter parameters and regularization method. And in most cases, this is a painstaking job. Wouldn't it be great to automate these tasks?

Fortunately, this is possible if we use the right tools! Specifically, we will see that Graph Convolutional Networks are a great framework to automatize the feature extraction method, by simply using gradient descent.

In this exercise, we will follow the same classification pipeline as in Exercise 5, but now, instead of hand-crafting our filter we will let `PyTorch` find the coefficients using gradient descent.

Let's start by constructing a `LaplacianPolynomial` model in `DGL`.

In [ ]:
class LaplacianPolynomial(nn.Module):
    def __init__(self,
                 in_feats,
                 out_feats,
                 k):
        super().__init__()
        self._in_feats = in_feats
        self._out_feats = out_feats
        self._k = k
        self.pol_weights = nn.Parameter(torch.Tensor(self._k + 1))
        self.logr_weights = nn.Parameter(torch.Tensor(in_feats, out_feats))
        dropout = 0.8
        self.dropout = nn.Dropout(p=dropout)
        self.reset_parameters()

    def reset_parameters(self):
        """Reinitialize learnable parameters."""
        torch.manual_seed(0)
        torch.nn.init.xavier_uniform_(self.logr_weights, gain=0.01)
        torch.nn.init.normal_(self.pol_weights, mean=0.0, std=1e-3)


    def forward(self, graph, feat):
        r"""Compute graph convolution.

        Notes
        -----
        * Input shape: :math:`(N, *, \text{in_feats})` where * means any number of additional
          dimensions, :math:`N` is the number of nodes.
        * Output shape: :math:`(N, *, \text{out_feats})` where all but the last dimension are
          the same shape as the input.

        Parameters
        ----------
        graph (DGLGraph) : The graph.
        feat (torch.Tensor): The input feature

        Returns
        -------
        (torch.Tensor) The output feature
        """
        feat = self.dropout(feat)
        graph = graph.local_var()

        # mult W first to reduce the feature size for aggregation.
        feat = torch.matmul(feat, self.logr_weights)

        result = self.pol_weights[0] * feat.clone()

        for i in range(1, self._k):
            graph.ndata['h'] = feat
            # Feat is not modified in place
            graph.update_all(fn.copy_src(src='h', out='m'),
                             fn.sum(msg='m', out='h'))
            feat = graph.in_degrees().float()[:, None] * feat - graph.ndata['h']
            result += self.pol_weights[i] * feat

        return result

    def extra_repr(self):
        """Set the extra representation of the module,
        which will come into effect when printing the model.
        """
        summary = 'in={_in_feats}, out={_out_feats}'
        summary += ', normalization={_norm}'
        return summary.format(**self.__dict__)

Once we have are model ready we just need to create a function that performs one step of our training loop, and another one that evaluates our model

In [ ]:
def train(model, g, features, labels, loss_fcn, train_mask, optimizer):
    model.train()
    # forward
    logits = model(g, features)  # only compute the train set
    loss = loss_fcn(logits[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss


def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)[mask] # only compute the evaluation set
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

Choose the training parameters...

In [ ]:
pol_order = 3

lr = 0.2
weight_decay = 5e-6
n_epochs = 1000

and let's train the classifier end to end. You should be able to obtain a test accuracy of more than 65%.

In [ ]:
graph = DGLGraph(cora.graph)

model = LaplacianPolynomial(in_feats, n_classes, pol_order)

loss_fcn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=lr,
                             weight_decay=weight_decay)

dur = []
for epoch in range(n_epochs):
    if epoch >= 3:
        t0 = time.time()
    loss = train(model, graph, features, labels, loss_fcn, train_mask, optimizer)

    if epoch >= 3:
        dur.append(time.time() - t0)

    acc = evaluate(model, graph, features, labels, val_mask)
    print("Epoch {:05d} | Time(s) {:.4f} | Train Loss {:.4f} | Val Accuracy {:.4f}". format(
            epoch, np.mean(dur), loss.item(), acc))

print()
acc = evaluate(model, graph, features, labels, test_mask)
print("Test Accuracy {:.4f}".format(acc))

Trained this way our GCN based on polynomials of the laplacian is a black box. Fortunately, however, the only difference between this shallow model and our previous classifier is the way we chose the filter coefficients.

Let's see what the network learnt...

In [ ]:
coeff_gcn = model.pol_weights.detach().numpy()
print(coeff_gcn)

To interpret the model we can plot the frequency response of the learned filter

In [ ]:
plt.plot(freq_grid, fir_graph_filter_response(coeff_gcn, freq_grid))

Based on your previous results, what type of graph filter would you say this is?

## Exercise 7

If everything is correct we should be able to use this filter to construct new hand-crafted filters and train a logistic regression model that achieves good accuracy on the training set. Let's do that!

Use the learned coefficients to train a new feature extractor

In [ ]:
graph_gcn = fir_graph_filter(coeff_gcn, laplacian)

Extract the new features by filtering the data

In [ ]:
features_gcn = graph_gcn @ features.numpy()

Train a logistic regression on this features

In [ ]:
train_features_gcn = features_gcn[train_mask,:]
train_labels = labels[train_mask]
val_features_gcn = features_gcn[val_mask,:]
val_labels = labels[val_mask]
test_features_gcn = features_gcn[test_mask,:]
test_labels = labels[test_mask]

log_reg_gcn = LogisticRegression(penalty='l2', multi_class="auto", solver="liblinear", C=1e4, fit_intercept=False, max_iter=1000)
log_reg_gcn.fit(train_features_gcn, train_labels)

Finally, let's evaluate this model

In [ ]:
train_acc = log_reg_gcn.score(train_features_gcn, train_labels)
val_acc = log_reg_gcn.score(val_features_gcn, val_labels)
test_acc = log_reg_gcn.score(test_features_gcn, test_labels)

print('Train accuracy {:.4f} | Validation accuracy {:.4f} | Test accuracy {:.4f}'.format(train_acc, val_acc, test_acc))